In [ ]:
from git import Repo

url = 'https://github.com/wbkd/react-flow'
!rm -rf ./clone
repo = Repo.clone_from(url, './clone')
branch = repo.head.reference

In [ ]:
from langchain.document_loaders import GitLoader

loader = GitLoader(
    repo_path="./clone", 
    branch=branch, 
    file_filter=lambda x: x.endswith(".js") or x.endswith(".jsx") or x.endswith(".ts") or x.endswith(".tsx") or x.endswith(".json") or x.endswith(".md") or x.endswith(".py"))
data = loader.load();

len(data)

# print(data[0])

In [ ]:
# data

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from os import environ

environ.get("PINECONE_ENVIRONMENT")

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
import pinecone
from langchain.vectorstores import Pinecone
from os import environ

pinecone.init(
    # api_key=environ.get("PINECONE_API_KEY"), 
    # environment=environ.get("PINECONE_ENVIRONMENT")
)
index_name = 'pinecone-hackathon-test'

if index_name not in pinecone.list_indexes():
    index= pinecone.create_index(
        name=index_name, 
        metric='cosine', 
        shards=1, 
        dimension=1536
    )
index = pinecone.Index(index_name=index_name)

index = Pinecone.from_documents(
    data, 
    embeddings, 
    index_name=index_name, 
    batch_size=10
)
# for x in range(len(data)):
#     print("adding: " + str(x))
#     if len(data[x].page_content) < 100000:
#         index = Pinecone.from_documents([data[x]], embeddings, index_name=index_name)
    
# pinecone.add_documents(data, index_name=index_name)
# index
index

In [ ]:
import pinecone 
pinecone.describe_index(index_name)

# index.describe_index_stats()